In [1]:
from dask.distributed import Client

In [2]:
client = Client('140.208.147.177:43131')

In [5]:
client

Client Scheduler: tcp://140.208.147.177:43131 Dashboard: http://140.208.147.177:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


### Using Xarray and net Files

In [4]:
%%time
import xarray as xr

#300yr OM4files, date 198801 - 200712

#chunk along dim = 'time'
OM4p25 = xr.open_mfdataset('/work/Nkeh.Boh/SST/NetOMfiles/OM4p25/*.nc')
OM4p25z = xr.open_mfdataset('/work/Nkeh.Boh/SST/NetOMfiles/OM4p25z/*.nc')
OM4p5nep = xr.open_mfdataset('/work/Nkeh.Boh/SST/NetOMfiles/OM4p5nep/*.nc')
OM4p5mle =xr.open_mfdataset('/work/Nkeh.Boh/SST/NetOMfiles/OM4p5mle/*.nc')
OM4p5 = xr.open_mfdataset('/work/Nkeh.Boh/SST/NetOMfiles/OM4p5/*.nc')
# 105GB < T_fileSize <110GB

OM4p25_av = OM4p25['tos'].mean(dim='time')#.load()
OM4p5nep_av = OM4p5nep['tos'].mean(dim='time')#.load()
OM4p25z_av = OM4p25z['tos'].mean(dim='time')#.load()
OM4p5_av = OM4p5['tos'].mean(dim='time')#.load()
OM4p5mle_av = OM4p5mle['tos'].mean(dim='time')#.load()


OM4p25_av.plot()
OM4p5nep_av.plot()
OM4p25z_av.plot()
OM4p25_av.plot()
OM4p5mle_av.plot()

KeyboardInterrupt: 

# import cartopy
# import cmocean
#import matplotlib.pyplot as plt
# import matplotlib.colors
# import matplotlib.colorbar
import xarray as xr
# import numpy

### 300yr files

#300yr OM4files, date 198801 - 200712

#chunk along dim = 'time'
OM4p25 = xr.open_mfdataset('/work/Nkeh.Boh/SST/NetOMfiles/OM4p25/*.nc')
OM4p25z = xr.open_mfdataset('/work/Nkeh.Boh/SST/NetOMfiles/OM4p25z/*.nc')
OM4p5nep = xr.open_mfdataset('/work/Nkeh.Boh/SST/NetOMfiles/OM4p5nep/*.nc')
OM4p5mle =xr.open_mfdataset('/work/Nkeh.Boh/SST/NetOMfiles/OM4p5mle/*.nc')
OM4p5 = xr.open_mfdataset('/work/Nkeh.Boh/SST/NetOMfiles/OM4p5/*.nc')
# 105GB < T_fileSize <110GB

#Grid files
# G25 = xr.open_dataset('/work/Nkeh.Boh/SST/NetOMfiles/G25/ocean_hgrid.nc')
# G5 = xr.open_dataset('/work/Nkeh.Boh/SST/NetOMfiles/G5/ocean_hgrid.nc')
# M25 = xr.open_dataset('/work/Nkeh.Boh/SST/NetOMfiles/M25/ocean_mask.nc')
# M5 = xr.open_dataset('/work/Nkeh.Boh/SST/NetOMfiles/M5/ocean_mask.nc')
#WOA files
# W25 = xr.open_dataset('/work/Nkeh.Boh/SST/NetOMfiles/W25/WOA05_ptemp_salt_monthly.v20141007.nc')
# W5 = xr.open_dataset('/work/Nkeh.Boh/SST/NetOMfiles/W05/WOA05_ptemp_salt_monthly.v2015.12.03.nc')


# OM4p5nep

## Averaging

# W25_ann = W25['ptemp'].mean(axis=(0,1))
# W5_ann = W5['ptemp'].mean(axis=(0,1))

why this variability; inefficiency in using all cores compared to the zarr format of thesame Dataset

# %%time
# OM4p25_av = OM4p25['tos'].mean(dim='time').load()
# OM4p5nep_av = OM4p5nep['tos'].mean(dim='time').load()
# OM4p25z_av = OM4p25z['tos'].mean(dim='time').load()
# OM4p5_av = OM4p5['tos'].mean(dim='time').load()
# OM4p5mle_av = OM4p5mle['tos'].mean(dim='time').load()

OM4p25_av = OM4p25['tos'].mean(dim='time')#.load()
OM4p5nep_av = OM4p5nep['tos'].mean(dim='time')#.load()
OM4p25z_av = OM4p25z['tos'].mean(dim='time')#.load()
OM4p5_av = OM4p5['tos'].mean(dim='time')#.load()
OM4p5mle_av = OM4p5mle['tos'].mean(dim='time')#.load()

## PLots

OM4p25_av.plot()

OM4p5nep_av.plot()

OM4p25z_av.plot()

OM4p25_av.plot()

OM4p5mle_av.plot()